In [4]:
from math import sqrt

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import numpy as np
import qutip as q
q.settings.colorblind_safe = True

import ipywidgets as widgets
from ipywidgets import GridspecLayout, interactive, Layout
from IPython.display import display

In [5]:
N = 15 # Cavity
M = 50 # Mirror

# Ladder operations of 
    # Cavity
a = q.tensor(q.destroy(N), q.identity(M))
ad = q.tensor(q.create(N), q.identity(M))

    # Mirror
b = q.tensor(q.identity(N), q.destroy(M))
bd = q.tensor(q.identity(N), q.create(M))

vec = np.linspace(-7.5, 7.5, 50)
xvec, yvec = vec, vec

In [6]:
def solve_qed(om_m=2*np.pi*1, om_c=2*np.pi*1, Om=2*np.pi**1, gamma=0, kappa=0, alpha=2, beta=2, n_th_a=0, t=np.linspace(0, 2*np.pi, 100)):
    """
    Solve the cavity QED system using qutip.mesolve.

    Parameters:
    om_a  : atomic transition frequency
    om_c  : cavity resonance frequency
    Om    : vacuum Rabi frequency (coupling strength)
    gamma : atomic decay rate
    alpha : displacement of basis state in the cavity 
    beta  : displacement of basis state in the atom 
    n_th_a: avg number of thermal bath excitation
    t     : array of time values at which to evaluate the state evolution
    """

    # Hamiltonian
    H = om_c * ad * a + om_m * bd * b - Om/2 * ad * a * (b + bd)
    
    # Initial state, atom in basis and cavity in coherent (can change)
    rho0 = q.tensor(q.coherent(N, alpha), q.coherent(M, beta))

    # Decay/dissipation
    c_ops = []

    # Cavity
    if gamma > 0: 
            # Photon annihilation
        c_ops.append(sqrt(gamma * (1 + n_th_a)) * a)
        
        if n_th_a > 0:
                # Photon creation
            c_ops.append(sqrt(gamma * n_th_a) * ad)
    
    # Mirror
    if kappa > 0: 
            # Photon annihilation
        c_ops.append(sqrt(kappa * (1 + n_th_a)) * b)

        if n_th_a > 0:
                # Photon creation
            c_ops.append(sqrt(kappa * n_th_a) * bd)


    # Master equation
    options = q.Options(store_states=True, rtol=1e-5, atol=1e-7, nsteps=10000)
    result = q.mesolve(H, rho0, t, c_ops, [], options=options)
    return result

In [7]:
# Linear entropy
def plot_entropy(result):
    plt.close()
    fig, axes = plt.subplots(1, 2, figsize=(6,3))
    
    entropy = [q.entropy_linear(q.ptrace(rho, 0)) for rho in result.states]
    
    axes[0].plot(result.times, entropy, color="tab:green")
    axes[0].set_title('Linear Entropy of Cavity')
    axes[0].set_xlabel(r'$t$', fontsize=12)
    axes[0].set_ylabel(r'$S$', fontsize=12)
    axes[0].set_ylim(0, 1)

    entropy = [q.entropy_linear(q.ptrace(rho, 1)) for rho in result.states]

    axes[1].plot(result.times, entropy, color="tab:orange")
    axes[1].set_title('Linear Entropy of Mirror')
    axes[1].set_xlabel(r'$t$', fontsize=12)
    axes[1].set_ylabel(r'$S$', fontsize=12)
    axes[1].set_ylim(0, 1)

    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False
    
    plt.tight_layout()
    plt.show()
    return

In [8]:
def plot_trunc(result, a, ad, b, bd):
    plt.close()
    fig, axes = plt.subplots(1, 2, figsize=(6,3))

    axes[0].plot(result.times, [q.expect(q.commutator(a, ad), s) for s in result.states], label='Cavity photon number', color="tab:green")
    axes[0].legend()
    axes[0].set_title(r'$[a, a^\dagger]$')
    axes[0].set_xlabel(r'$t$', fontsize=12)
    axes[0].set_ylim(0, 1.1)
    axes[0].legend()
    
    axes[1].plot(result.times, [q.expect(q.commutator(b, bd), s) for s in result.states], label='Mirror photon number', color="tab:orange")
    axes[1].legend()
    axes[1].set_title(r'$[b, b^\dagger]$')
    axes[1].set_xlabel(r'$t$', fontsize=12)
    axes[1].set_ylim(0, 1.1)
    axes[1].legend()

    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False
    
    plt.tight_layout()
    plt.show()
    return

In [9]:
def plot_wigner_fock_cav():
    plt.close()

    k_vec = np.array([0.5, 1/np.sqrt(6), 1/(2*np.sqrt(2))])*2
    result = [solve_qed(Om = 2*np.pi*k) for k in k_vec]
    
    x = result[0].times
    step = [16]
    
    fig, axes = plt.subplots(len(k_vec), 2, figsize=(6,3*len(k_vec)))
        
    for i in range(len(k_vec)):
        j = step[0]
        tid = x[j]
        
        # Plot the subsystem of the Cavity
        rho_sub_c = [q.ptrace(rho, 0) for rho in result[i].states] # States of cavity subsystem
        
            # Fock distribution
        q.plot_fock_distribution(rho_sub_c[j], ax=axes[i, 1], color="tab:green")
        #axes[i, 1].set_title(r'At time: ' + r"$t=$" + f"{tid:.2f}", x=2)
        axes[i, 1].set_ylim(0, 1.01)
        axes[i, 1].set_xticks(np.arange(0,N+1, 5))

            # Wigner function
        q.plot_wigner(rho_sub_c[j], xvec=xvec, yvec=yvec, ax=axes[i, 0], cmap="RdBu_r")
        axes[i, 0].set_title(r'At ' + r"$t=$" + f"{tid:.2f}" + r", $\Omega$" + f"={k_vec[i]/2:.2f}" , x=1.1, fontsize = 14)

    fig.suptitle('Wigner func. and Fock dist. at various times \n for the Cavity', fontsize=16)
    
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False
    
    plt.tight_layout()
    plt.show()
    return

In [10]:
def plot_wigner_fock_mir():
    plt.close()

    result = solve_qed(Om = 2*np.pi*1)
    
    x = result.times
    step = [0, 4, 8, 12, 16]
    
    fig, axes = plt.subplots(len(step), 2, figsize=(6,3*len(step)))
        
    for i in range(len(step)):
        j = step[i]
        tid = x[j]
        
        # Plot the subsystem of the Cavity
        rho_sub_m = [q.ptrace(rho, 1) for rho in result.states] # States of cavity subsystem
        
            # Fock distribution
        q.plot_fock_distribution(rho_sub_m[j], ax=axes[i, 1], color="tab:orange")
        #axes[i, 1].set_title(r'At time: ' + r"$t=$" + f"{tid:.2f}", x=2)
        axes[i, 1].set_ylim(0, 1.01)
        axes[i, 1].set_xticks(np.arange(0,M+1, 5))
        axes[i, 1].spines[['right', 'top']].set_visible(False)
        
            # Wigner function
        q.plot_wigner(rho_sub_m[j], xvec=xvec, yvec=yvec, ax=axes[i, 0], cmap="RdBu_r")
        axes[i, 0].set_title(r'At ' + r"$t=$" + f"{tid:.2f}" , x=1.1, fontsize = 14)

    fig.suptitle('Wigner func. and Fock dist. at various times \n for the Mirror', fontsize=16)
    
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False
    
    plt.tight_layout()
    plt.show()
    return

In [11]:
def plot_rabi():
    plt.close()

    result = solve_qed()
    
    fig, axes = plt.subplots(1, 1, figsize=(6,3))

    z1_sub = [q.expect(ad*a, s) for s in result.states]     # Expect cavity occ. prob.
    z2_sub = [q.expect(bd*b, s) for s in result.states]     # Expect mirror occ. prob.

        # Plot the subsystem of the Cavity and Atom
    axes.plot(result.times, z1_sub, '-', color="tab:green", label=r"Cavity: $E(a^\dagger a, \phi_c)$")
    axes.plot(result.times, z2_sub, '-', color="tab:orange", label=r"Mirror: $E(b^\dagger b, \phi_c)$")
    axes.set_xlim(0, 6)
    axes.set_ylim(2, 8)
    #axes.set_title(r"At $\Omega$"+f"= {k_vec[i]/2:.1f}")
    axes.set_xlabel(r'$\rm{Time}$', fontsize=12)
    axes.set_ylabel(r'$\rm{Occ. prob.}$', fontsize=12)
    axes.legend(loc="upper right")
    axes.spines[['right', 'top']].set_visible(False)

    fig.suptitle(r'Rabi oscilation for various $\Omega$ at $0K$', fontsize=16)
    
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False

    plt.tight_layout()
    plt.show()
    return

In [12]:
def plot_(result, xvec=None, yvec=None, n_th_a=0, alpha=2, beta=2, method='clenshaw', projection='2d',
                g=sqrt(2), sparse=False, parfor=False, *,
                cmap=None, colorbar=False, fig=None, ax=None):
    
    plt.close()
    
    fontsize = 12

    offset=0.5
    
    # Create the figure and axes
    fig = plt.figure(figsize=(7,7))
    axes1 = plt.subplot(313)
    axes2 = plt.subplot(321)
    axes3 = plt.subplot(322)
    axes4 = plt.subplot(323)
    axes5 = plt.subplot(324)
    
    plt.subplots_adjust(wspace=.4)
    plt.subplots_adjust(hspace=.7)
    
    # Set up the plot's initial state
        # Rabi oscillation
    axes1.set_xlim(0, 2*np.pi)
    axes1.set_title(f'Rabi Osc. at {n_th_a}K')
    axes1.set_xlabel(r'$\rm{time}$', fontsize=fontsize)
    axes1.set_ylabel(r'$\rm{Occ. prob.}$', fontsize=fontsize)
    axes1.spines[['right', 'top']].set_visible(False)

        # Wigner fucntion, mirror
    axes2.set_title('Wigner Func. of Mirror')
    axes2.set_xlabel(r'$\rm{Re}(\alpha)$', fontsize=fontsize)
    axes2.set_ylabel(r'$\rm{Im}(\alpha)$', fontsize=fontsize)

        # Fock distribution, mirror
    axes3.set_ylim(0, 1)
    axes3.set_xlim(np.arange(-.5 + offset, M + offset, M))
    axes3.set_title('Fock Dist. of Mirror')
    axes3.set_xlabel('Fock number', fontsize=fontsize)
    axes3.set_ylabel('Occ. prob.', fontsize=fontsize)
    axes3.spines[['right', 'top']].set_visible(False)
        
        # Wigner fucntion, cavity
    axes4.set_title('Wigner Func. of Cavity')
    axes4.set_xlabel(r'$\rm{Re}(\alpha)$', fontsize=fontsize)
    axes4.set_ylabel(r'$\rm{Im}(\alpha)$', fontsize=fontsize)
            
        # Fock distribution, cavity
    axes5.set_ylim(0, 1)
    axes5.set_xlim(np.arange(-.5 + offset, N + offset, N))
    axes5.set_title('Fock Dist. of Cavity')
    axes5.set_xlabel('Fock number', fontsize=fontsize)
    axes5.set_ylabel('Occ. prob.', fontsize=fontsize)
    axes5.spines[['right', 'top']].set_visible(False)
    
    
    if colorbar:
        shrink = 0.75
        
        cax_c, kw = mpl.colorbar.make_axes(axes2, shrink=shrink, pad=.05)
        cax_m, kw = mpl.colorbar.make_axes(axes4, shrink=shrink, pad=.05)
    
    # Create a list to hold the artists for each frame
    artists = []
    
    # Generate the data and artists for all frames
    xdata1, ydata1 = [], []
    xdata2, ydata2 = [], []

    rhos_c = [q.ptrace(rho, 0) for rho in result.states]
    rhos_m = [q.ptrace(rho, 1) for rho in result.states]

    exps_c = [q.expect(ad*a, rho) for rho in result.states]
    exps_m = [q.expect(bd*b, rho) for rho in result.states]
    
    x = result.times
    
    for frame in np.arange(0,len(x),1):
        frame = int(frame)

        rho_c = rhos_c[frame]
        rho_m = rhos_m[frame]

        exp_c = exps_c[frame]
        exp_m = exps_m[frame]

        # Append new data for each line
        xdata1.append(x[frame])
        ydata1.append(exp_c)
    
        xdata2 = xdata1
        ydata2.append(exp_m)
    
        # Plot for the current frame
            # Occupation prob.
        ln1, = axes1.plot(xdata1, ydata1, '-', color="tab:green")
        ln2, = axes1.plot(xdata2, ydata2, '-', color="tab:orange")

            # Wigner        
        W0_c = q.wigner(rho_c, xvec, yvec, method=method,
                        g=g, sparse=sparse, parfor=parfor)
        W0_m = q.wigner(rho_m, xvec, yvec, method=method,
                        g=g, sparse=sparse, parfor=parfor)
    
        
        W_c, yvec = W0_c if isinstance(W0_c, tuple) else (W0_c, yvec)
        W_m, yvec = W0_m if isinstance(W0_m, tuple) else (W0_m, yvec)
        
        wlim_c = abs(W_c).max()
        wlim_m = abs(W_m).max()
        
        norm_c = mpl.colors.Normalize(-wlim_c, wlim_c)
        norm_m = mpl.colors.Normalize(-wlim_m, wlim_m)

        cbar_c = mpl.colorbar.ColorbarBase(cax_c, cmap=cmap, norm=norm_c)
        cbar_m = mpl.colorbar.ColorbarBase(cax_m, cmap=cmap, norm=norm_m)
    # Time consuming - ...
        cf_m = axes2.contourf(xvec, yvec, W_m, 100, norm=norm_m, cmap=cmap)
        cf_c = axes4.contourf(xvec, yvec, W_c, 100, norm=norm_c, cmap=cmap)

            # Fock dist.
        F_c = np.real(rho_c.diag())
        F_m = np.real(rho_m.diag())
    # Time consuming - ...
        bars_m = axes3.bar(np.arange(offset, offset + M) - .4, F_m,
               color="tab:orange", alpha=0.6, width=0.8)
        bars_c = axes5.bar(np.arange(offset, offset + N) - .4, F_c,
               color="tab:green", alpha=0.6, width=0.8)

        bars = np.append(bars_c, bars_m)
        
        # Add the artists for the current frame to the list
        artists.append(np.append(np.array([ln1, ln2, cf_c, cf_m]), bars))

    # Create the animation
    anim = animation.ArtistAnimation(fig, artists, interval=len(x), blit=True)
    
    ln1.set_label('Cavity')
    ln2.set_label('Mirror')
    axes1.legend(loc='upper right', fontsize='small')
    
    # The window not (still covers x-axis label)
    fig.canvas.layout.width = '7in'
    fig.canvas.layout.height= '8.1in'
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False
    
    
    # To display the animation, you would typically use plt.show()
    plt.show()
    return fig, anim

In [13]:
def sliders(N, M):
    
    # Sliders
    slider_gamma = widgets.FloatSlider(value=0,
                                     min=0.,
                                     max=1.,
                                     step=0.01,
                                     description='gamma:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='.1f')
    
    slider_kappa = widgets.FloatSlider(value=0,
                                     min=0.,
                                     max=1.,
                                     step=0.01,
                                     description='kappa:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='.1f')
    
    slider_n_th_a = widgets.IntSlider(value=0,
                                     min=0,
                                     max=300,
                                     step=1,
                                     description='temp.:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='d')
    
    slider_alpha = widgets.IntSlider(value=2,
                                     min=0,
                                     max=N,
                                     step=1,
                                     description='alpha:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='d')
    
    slider_beta = widgets.IntSlider(value=2,
                                     min=0,
                                     max=M,
                                     step=1,
                                     description='beta:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='d')
    
    slider_om_m = widgets.FloatSlider(value=1,
                                     min=0.,
                                     max=1.,
                                     step=.1,
                                     description='om_m:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='.1f')
    
    slider_om_c = widgets.FloatSlider(value=1,
                                     min=0.,
                                     max=1.,
                                     step=.1,
                                     description='om_c:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='.1f')
    
    slider_Om = widgets.FloatSlider(value=1,
                                     min=0.,
                                     max=1.,
                                     step=.1,
                                     description='Om:',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True,
                                     readout_format='.1f')
    return slider_gamma, slider_kappa, slider_n_th_a, slider_alpha, slider_beta, slider_om_m, slider_om_c, slider_Om

def grid_layout(widget_list):
    # Use GridspecLayout to create the 2-column grid
    # The number of rows is calculated based on the number of widgets
    n_widgets = len(widget_list)
    n_cols = 2
    n_rows = (n_widgets + n_cols - 1) // n_cols
    
    grid = GridspecLayout(n_rows, n_cols, height='auto')
    
    # Place the widgets into the grid
    for i, widget in enumerate(widget_list):
        row = i // n_cols
        col = i % n_cols
        grid[row, col] = widget
    return grid